# Регулярные выражения

<img width='450px' src="images/use-regex.jpg"></img>

Некоторые люди, когда сталкиваются с проблемой, думают:

    «Я знаю, я решу её с помощью регулярных выражений.»

Теперь у них **ДВЕ** проблемы.

## Что почитать?

1. Хороший [«быстрый старт»](https://tproger.ru/translations/regular-expression-python/) с задачками от TProger.
2. Отличный [туториал](https://habr.com/post/349860/) с задачками на Хабре.
3. [Документация](https://docs.python.org/3/library/re.html) к стандартной библиотеке.

In [1]:
import re

# Структура URL

http://www.example.com:80/path/to/myfile.html?key1=value1&key2=value2#SomewhereInTheDocument

<img width='480px' src="https://mdn.mozillademos.org/files/8013/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8015/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8017/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8019/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8021/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8023/mdn-url-protocol@x2.png"></img>

https://developer.mozilla.org/ru/docs/Learn/Understanding_URLs

# Работа с JSON

In [2]:
import requests

[Документация](http://docs.python-requests.org/) по библиотеке requests.

In [3]:
import json

Prepare token: https://developer.spotify.com/console/get-track/?id=3n3Ppam7vgaVa1iaRUc9Lp

In [225]:
with open('spotify-api-token.key', mode='r') as f_key:
    spotify_api_token = f_key.readline()
    

class SpotifyAPI:    
    def __init__(self, token):
        self._token = token

    def get_track_info(self, track_id, market='US'):
        # посылаем get запрос, указываем аргументы и заголовки
        ret = requests.get("https://api.spotify.com/v1/tracks/{}".format(track_id),
                           params={'market': market,},
                           headers={'Authorization': f'Bearer {self._token}'})
        return ret


spotify = SpotifyAPI(spotify_api_token)
r = spotify.get_track_info('3n3Ppam7vgaVa1iaRUc9Lp')
r.status_code

ConnectionError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/tracks/3n3Ppam7vgaVa1iaRUc9Lp?market=US (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fadee203b38>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [226]:
r.headers

AttributeError: 'bytes' object has no attribute 'headers'

In [227]:
r.content

AttributeError: 'bytes' object has no attribute 'content'

In [228]:
answer = json.loads(r.content)
type(answer)

AttributeError: 'bytes' object has no attribute 'content'

In [229]:
answer = r.json()
type(answer)

AttributeError: 'bytes' object has no attribute 'json'

In [230]:
answer_cut = {
    'artist':   answer['artists'][0]['name'],
    'name':     answer['name'],
    'duration': answer['duration_ms'] / 1000.0,
    'href':     answer['href'],
    'id':       answer['id'],
}
answer_cut

NameError: name 'answer' is not defined

In [231]:
# компактно сериализуем json в строку
json.dumps(answer_cut, separators=(',', ':'))

NameError: name 'answer_cut' is not defined

In [232]:
# pretty print для json
print(json.dumps(answer_cut, indent=4, sort_keys=True))

NameError: name 'answer_cut' is not defined

In [233]:
with open('/tmp/answer_cut.json', mode='w') as f_json:
    json.dump(answer_cut, f_json)

NameError: name 'answer_cut' is not defined

In [234]:
!cat /tmp/answer_cut.json

In [235]:
!cat /tmp/answer_cut.json | python -m json.tool

No JSON object could be decoded


In [236]:
with open('/tmp/answer_cut.json', mode='r') as f_json:
    answer_cut = json.load(f_json)
answer_cut

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## А можно ли как-то ускорить процесс обкачки?

Скачивание данных через библиотеку requsets – I/O операция, поэтому большую часть времени процессор не выполняет никаких задач, потому что ждёт, пока данные придут по сети.

In [4]:
from tqdm import tqdm

In [238]:
%%time

n_post_first, n_post_final = 349000, 349200

r = [requests.get('https://habrahabr.ru/post/{}/'.format(post_id))
     for post_id in tqdm(range(n_post_first, n_post_final, 2))]


  0%|          | 0/100 [00:00<?, ?it/s]

ConnectionError: HTTPSConnectionPool(host='habrahabr.ru', port=443): Max retries exceeded with url: /post/349000/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fadee3f5f98>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [5]:
from multiprocessing.dummy import Pool as ThreadPool

In [6]:
%%time

n_post_first, n_post_final = 349000, 349200

t = ThreadPool(16)
r = t.map(lambda post_id: requests.get('https://habrahabr.ru/post/{}/'.format(post_id)),
          range(n_post_first, n_post_final, 2))
t.close()
t.join()

CPU times: user 6.17 s, sys: 425 ms, total: 6.59 s
Wall time: 10.9 s


In [10]:
r[0].text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Отчёт Центра мониторинга информационной безопасности за II полугодие 2017 года / Блог компании Перспективный мониторинг / Хабр</title>\n\n  <meta name="description" content="C начала 2018 года мы переходим на полугодовой цикл публикации отчётов Центра мониторинга нашей компании. Данный отчёт охватывает период с июля по декабрь 2017 года.\r\n\r\n\r\n\r\nПод катом о том, почему..." />\n\n  <meta name="keywords" content="soc, security operation center, информационная безопасность, инциденты ИБ, киберугрозы" />\n\n  <meta property="fb:app_id" content="444736788986613" />\n<meta property="og:locale" content="ru_RU" />\n<meta property="og:type" content="article" />\n<meta property="fb:pages" content="472597926099084"/>\n<meta property="og:url" content="https://habr.com/company/pm/blog/349000/" />\n<meta 

О том, какие функции в Python можно помещать в потоки вы узнаете на лекции, посвященной многопоточке.

# Парсинг HTML-страничек

## robots.txt

1. Валидаторы:
    * https://webmaster.yandex.ru/tools/robotstxt/
    * https://technicalseo.com/seo-tools/robots-txt/
2. Все о robots.txt:
    * https://help.mail.ru/webmaster/indexing/robots.txt
    * https://yandex.ru/support/webmaster/controlling-robot/

## HTMLParser

In [120]:
from html.parser import HTMLParser

In [121]:
film_url = 'https://www.kinopoisk.ru/film/garri-potter-i-taynaya-komnata-2002-688/'
film_html = requests.get(film_url).text

In [122]:
class KinoPoiskParser(HTMLParser):
    """
    Парсер страниц КиноПоиска с фильмами.

    Извлекает следующую информацию:
        1. название;
        2. оригинальное название;
        3. страны производители;
        4. длительность;
        5. описание;
        6. рейтинг.
    """

    def __init__(self):
        super().__init__()
        self.film_info = dict()
        self.status = None
        
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)
        self.status = None
        
        # HTML Parser не умеет сплитить атрибуты
        if 'class' in attrs:
            attrs['class'] = attrs['class'].split()
        else:
            attrs['class'] = []
        
        if tag == 'h1' and 'moviename-big' in attrs['class']:
            # 1. Название фильма
            self.status = 'title'
        elif tag == 'span':
            if attrs.get('itemprop', '') == 'alternativeHeadline':
                # 2. Оригинальное название фильма
                self.status = 'title-original'
            elif 'rating_ball' in attrs['class']:
                # 6. Рейтинг фильма
                self.status = 'rating'
        elif tag == 'div':
            if 'flag' in attrs['class']:
                # 3. Страны производители
                if 'countries' not in self.film_info:
                    self.film_info['countries'] = []
                
                country = attrs.get('title')
                if country:
                    self.film_info['countries'].append(country)
            elif attrs.get('itemprop') == 'description':
                # 5. Описание фильма
                self.status = 'description'
        elif tag == 'td' and 'time' in attrs['class'] and \
                attrs.get('id', '') == 'runtime':
            # 4. Длительность фильма
            self.status = 'time'
            
    def handle_data(self, data):
        data = data.strip()
        
        if not data:
            return
        
        if self.status is not None:
            if self.status == 'rating':
                # приводим рейтинг к float
                data = float(data)
            elif self.status == 'time':
                # форматируем дату
                data = re.search('\d+', data)
                data = int(data.group(0))
                data = "{:02}:{}".format(data // 60, data % 60)
            else:
                # в остальных случаях убираем лишние пробелы
                data = re.sub('\s+', ' ', data)
            self.film_info[self.status] = data 
            
    def clear():
        # очищаем информацию о фильме, перед обработкой
        # новой страницы
        self.film_info = {}

In [123]:
parser = KinoPoiskParser()

In [124]:
parser.feed(film_html)
parser.film_info

{'title': 'Гарри Поттер и Тайная комната',
 'title-original': 'Harry Potter and the Chamber of Secrets',
 'countries': ['Великобритания', 'США', 'Германия'],
 'time': '02:41',
 'description': 'Гарри Поттер переходит на второй курс Школы чародейства и волшебства Хогвартс. Эльф Добби предупреждает Гарри об опасности, которая поджидает его там, и просит больше не возвращаться в школу.',
 'rating': 8.03}

## Beautiful Soup
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

<img width = '330px' src="images/bsoup.jpg">

In [20]:
from bs4 import BeautifulSoup

In [126]:
soup = BeautifulSoup(film_html, 'html.parser')

### Соберем основную информацию о фильме

In [127]:
film_info = {
    'title': soup.find('h1', class_='moviename-big').text,
    'title-original': soup.find('span', itemprop='alternativeHeadline').text,
    'countries': list(map(lambda e: e.attrs['title'], soup.find_all('div', class_='flag'))),
    'time': re.search('\d\d:\d\d', soup.find('td', class_='time').text).group(0),
    'rating': float(soup.select_one('.rating_ball').text) # поддержка CSS-селекторов
}

film_info

{'title': 'Гарри Поттер и Тайная комната',
 'title-original': 'Harry Potter and the Chamber of Secrets',
 'countries': ['Великобритания', 'США', 'Германия'],
 'time': '02:41',
 'rating': 8.03}

In [128]:
desc = soup.find('div', itemprop='description')
desc

<div class="brand_words film-synopsys" itemprop="description">Гарри Поттер переходит на второй курс Школы чародейства и волшебства Хогвартс. Эльф Добби предупреждает Гарри об опасности, которая поджидает его там, и просит больше не возвращаться в школу.<br/><br/>Юный волшебник не следует совету эльфа и становится свидетелем таинственных событий, разворачивающихся в Хогвартсе. Вскоре Гарри и его друзья узнают о существовании Тайной Комнаты и сталкиваются с новыми приключениями, пытаясь победить темные силы.</div>

In [129]:
desc.text

'Гарри Поттер переходит на\xa0второй курс Школы чародейства и\xa0волшебства Хогвартс. Эльф Добби предупреждает Гарри об\xa0опасности, которая поджидает его\xa0там, и\xa0просит больше не\xa0возвращаться в\xa0школу.Юный волшебник не\xa0следует совету эльфа и\xa0становится свидетелем таинственных событий, разворачивающихся в\xa0Хогвартсе. Вскоре Гарри и\xa0его друзья узнают о\xa0существовании Тайной Комнаты и\xa0сталкиваются с\xa0новыми приключениями, пытаясь победить темные силы.'

In [130]:
for br in desc.find_all("br"):
    br.replace_with("\n")
desc

<div class="brand_words film-synopsys" itemprop="description">Гарри Поттер переходит на второй курс Школы чародейства и волшебства Хогвартс. Эльф Добби предупреждает Гарри об опасности, которая поджидает его там, и просит больше не возвращаться в школу.

Юный волшебник не следует совету эльфа и становится свидетелем таинственных событий, разворачивающихся в Хогвартсе. Вскоре Гарри и его друзья узнают о существовании Тайной Комнаты и сталкиваются с новыми приключениями, пытаясь победить темные силы.</div>

In [131]:
film_info['description'] = re.sub('\s+', ' ', desc.text)
film_info

{'title': 'Гарри Поттер и Тайная комната',
 'title-original': 'Harry Potter and the Chamber of Secrets',
 'countries': ['Великобритания', 'США', 'Германия'],
 'time': '02:41',
 'rating': 8.03,
 'description': 'Гарри Поттер переходит на второй курс Школы чародейства и волшебства Хогвартс. Эльф Добби предупреждает Гарри об опасности, которая поджидает его там, и просит больше не возвращаться в школу. Юный волшебник не следует совету эльфа и становится свидетелем таинственных событий, разворачивающихся в Хогвартсе. Вскоре Гарри и его друзья узнают о существовании Тайной Комнаты и сталкиваются с новыми приключениями, пытаясь победить темные силы.'}

### Извлечем таблицу

In [132]:
table = soup.find('table', class_='info')
rows = table.find_all('tr')

In [133]:
data = [list(map(lambda x: x.text, row.find_all('td'))) for row in rows]
data

[['год', '\n2002\n'],
 ['страна', '\nВеликобритания, США, Германия\n'],
 ['слоган', '«Зло вернулось в Хогвартс»'],
 ['режиссер', 'Крис Коламбус'],
 ['сценарий', 'Стивен Кловз, Дж.К. Роулинг'],
 ['продюсер', 'Дэвид Хейман, Майкл Барнатан, Дэвид Баррон, ...'],
 ['оператор', 'Роджер Прэтт'],
 ['композитор', 'Джон Уильямс'],
 ['художник', 'Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Бартоломью, ...'],
 ['монтаж', 'Питер Хонесс'],
 ['жанр', '\nфэнтези, детектив, приключения, семейный, ...\nслова\n'],
 ['бюджет', '\n$100\xa0000\xa0000\n'],
 ['маркетинг', '\n$50\xa0000\xa0000\n'],
 ['сборы в США',
  '\n\n$261\xa0988\xa0482\n\n\n var is_usa_box_popup = true; \n'],
 ['сборы в мире',
  '\n\n+\xa0$616\xa0991\xa0152\xa0=\xa0$878\xa0979\xa0634\nсборы\n\n var is_world_box_popup = false; \n'],
 ['сборы в России',
  '\n\n$8\xa0000\xa0000\n\n\n var is_rus_box_popup = true; \n'],
 ['зрители',
  '\n\xa0\xa045.1 млн,\xa0\xa0\xa0\xa0\xa0\xa013.9 млн,\xa0\xa0\xa0\xa0\xa0\xa012.2 млн, ...\n'],
 ['премьера (мир)', 

In [134]:
data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('td')))
        for row in rows]
data

[['год', '2002'],
 ['страна', 'Великобритания, США, Германия'],
 ['слоган', '«Зло вернулось в Хогвартс»'],
 ['режиссер', 'Крис Коламбус'],
 ['сценарий', 'Стивен Кловз, Дж.К. Роулинг'],
 ['продюсер', 'Дэвид Хейман, Майкл Барнатан, Дэвид Баррон, ...'],
 ['оператор', 'Роджер Прэтт'],
 ['композитор', 'Джон Уильямс'],
 ['художник', 'Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Бартоломью, ...'],
 ['монтаж', 'Питер Хонесс'],
 ['жанр', 'фэнтези, детектив, приключения, семейный, ... слова'],
 ['бюджет', '$100 000 000'],
 ['маркетинг', '$50 000 000'],
 ['сборы в США', '$261 988 482 var is_usa_box_popup = true;'],
 ['сборы в мире',
  '+ $616 991 152 = $878 979 634 сборы var is_world_box_popup = false;'],
 ['сборы в России', '$8 000 000 var is_rus_box_popup = true;'],
 ['зрители', '45.1 млн, 13.9 млн, 12.2 млн, ...'],
 ['премьера (мир)', '3 ноября 2002, ...'],
 ['премьера (РФ)', '26 декабря 2002, «Каро-Премьер»'],
 ['релиз на DVD', '22 апреля 2003, «Premier Digital», ...'],
 ['релиз на Blu-ray', '20 янв

In [135]:
for row in rows:
    for script in row.find_all("script"):
        script.replace_with('')
        
data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('td')))
        for row in rows]
data

[['год', '2002'],
 ['страна', 'Великобритания, США, Германия'],
 ['слоган', '«Зло вернулось в Хогвартс»'],
 ['режиссер', 'Крис Коламбус'],
 ['сценарий', 'Стивен Кловз, Дж.К. Роулинг'],
 ['продюсер', 'Дэвид Хейман, Майкл Барнатан, Дэвид Баррон, ...'],
 ['оператор', 'Роджер Прэтт'],
 ['композитор', 'Джон Уильямс'],
 ['художник', 'Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Бартоломью, ...'],
 ['монтаж', 'Питер Хонесс'],
 ['жанр', 'фэнтези, детектив, приключения, семейный, ... слова'],
 ['бюджет', '$100 000 000'],
 ['маркетинг', '$50 000 000'],
 ['сборы в США', '$261 988 482'],
 ['сборы в мире', '+ $616 991 152 = $878 979 634 сборы'],
 ['сборы в России', '$8 000 000'],
 ['зрители', '45.1 млн, 13.9 млн, 12.2 млн, ...'],
 ['премьера (мир)', '3 ноября 2002, ...'],
 ['премьера (РФ)', '26 декабря 2002, «Каро-Премьер»'],
 ['релиз на DVD', '22 апреля 2003, «Premier Digital», ...'],
 ['релиз на Blu-ray', '20 января 2009, «Юниверсал Пикчерс Рус», ...'],
 ['возраст', 'зрителям, достигшим 12 лет'],
 ['рей

In [136]:
import pandas as pd

In [137]:
# Внимание! Из таблицы все равно придется убрать скрипты

df = pd.read_html(str(table))[0]
df

,0,1
0,год,2002
1,страна,"Великобритания, США, Германия"
2,слоган,«Зло вернулось в Хогвартс»
3,режиссер,Крис Коламбус
4,сценарий,"Стивен Кловз, Дж.К. Роулинг"
5,продюсер,"Дэвид Хейман, Майкл Барнатан, Дэвид Баррон, ..."
6,оператор,Роджер Прэтт
7,композитор,Джон Уильямс
8,художник,"Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Бартолом..."
9,монтаж,Питер Хонесс


### Пройдемся по спискам

In [138]:
actors, actors_dub = soup.find('div', id='actorList').find_all('ul')

In [139]:
[row.text for row in actors.children if row.text != '...']

['Дэниэл Рэдклифф',
 'Руперт Гринт',
 'Эмма Уотсон',
 'Том Фелтон',
 'Кеннет Брана',
 'Бонни Райт',
 'Алан Рикман',
 'Ричард Харрис',
 'Мэгги Смит',
 'Робби Колтрейн']

In [140]:
[row.text for row in actors_dub.children if row.text != '...']

['Николай Быстров',
 'Ольга Сирина',
 'Лина Иванова',
 'Александр Скрывля',
 'Вадим Андреев']

### Обкачаем картинки

In [141]:
film_photos_html = requests.get(
    'https://www.kinopoisk.ru/film/garri-potter-i-taynaya-komnata-2002-688/stills/').content

In [142]:
soup = BeautifulSoup(film_photos_html, 'html.parser')

In [143]:
list(map(lambda s: s.attrs['src'], soup.find('table', class_='fotos').findAll("img")))[:10]

['https://st.kp.yandex.net/images/kadr/sm_2094224.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005184.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005183.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005182.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005181.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558456.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558455.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558454.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558453.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558452.jpg']

## Заголовки

In [144]:
r = requests.get('https://www.dafont.com/')
len(r.text)

0

https://pypi.org/project/fake-useragent/

In [145]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}

r = requests.get('https://www.vk.com/', headers=headers)
len(r.text)

29231

## Cookie

Cookie (куки) — это небольшие текстовые файлы, в которые браузер записывает данные с посещенных вами сайтов. Файлы cookie позволяют сайтам «запоминать» своих посетителей, например, чтобы каждый раз не переспрашивать их логин и пароль.

chrome://settings/cookies/detail?site=sphere.mail.ru (sessionid_gtp)

In [151]:
r = requests.get('https://sphere.mail.ru/people/', params={'q': 'BD-31'})
r.ok

True

In [152]:
soup = BeautifulSoup(r.content, 'html.parser')

table = soup.find('div', class_='people-list')
table is not None

False

In [159]:
with open('sphere-cookies.json', mode='r') as f_cookies:
    cookies = json.load(f_cookies)
    
r = requests.get('https://sphere.mail.ru/people/', params={'q': 'BD-11'}, cookies=cookies)
r.ok

True

In [160]:
soup = BeautifulSoup(r.content, 'html.parser')

table = soup.find('div', class_='people-list')
table is not None

True

In [161]:
from urllib.parse import urlparse

In [164]:
rows = table.find('table', class_='table-users').find('tbody').find_all('tr')

users = [{'name': row.find('p', class_='realname').text,
          'nick': urlparse(row.find('p', class_='username').find('a').attrs['href']).\
              path.split('/')[2],
          'type': row.find('span', class_='user-group').text,
          'power': row.find('td', class_='cell-skill').text,
          'rating': row.find('td', class_='cell-rating').text.strip()}
         for row in rows]

df = pd.DataFrame(users)
df

,name,nick,power,rating,type
0,Александр Чернышёв,a.chernyishyov,"0,10",3.16,Студент
1,Полина Волосникова,p.volosnikova,"0,00",2.94,Студент
2,Ирина Авакянц,i.avakyants,"0,00",2.10,Студент
3,Николай Слепов,n.slepov,"0,00",1.48,Студент
4,Семен Кушелев,se.kushelev,"0,00",1.27,Студент
5,Игорь Петров,ig.petrov,"0,00",1.26,Студент
6,Юлия Недоливко,yu.nedolivko,"0,00",1.26,Студент
7,Павел Мамаев,p.mamaev,"0,00",1.26,Студент
8,Александр Соболев,al.sobolev,"0,01",1.26,Студент
9,Сергей Шадрин,s.shadrin,"0,01",1.26,Студент


## Proxy

In [169]:
r = requests.get('https://yandex.ru/internet/')

soup = BeautifulSoup(r.content, 'html.parser')

ip = soup.find('li', class_='client__item_type_ipv4').\
    find('div', class_='client__desc').text
loc = soup.find('li', class_='client__item_type_location').\
    find('div', class_='client__desc').text
loc = re.sub('\s?Настроить', '', loc)

ip, loc

('188.44.42.210', 'Москва')

http://spys.one/proxys/DE/

In [170]:
proxies = { 'https': '138.68.73.59:32574' }
r = requests.get('https://yandex.ru/internet/', proxies=proxies)

soup = BeautifulSoup(r.content, 'html.parser')

ip = soup.find('li', class_='client__item_type_ipv4').\
    find('div', class_='client__desc').text
loc = soup.find('li', class_='client__item_type_location').\
    find('div', class_='client__desc').text
loc = re.sub('\s?Настроить', '', loc)

ip, loc

('138.68.73.59', 'Берлин')

## Selenium
http://selenium-python.readthedocs.io/

https://kreisfahrer.gitbooks.io/selenium-webdriver/

In [183]:
url_format = "http://www.banki.ru/banks/map/{city}/#/!b1:{bank_ids}!s3:{bank_types}!s4:list!p1:{page}"

In [184]:
url_params = {
    'city': 'moskva_i_oblast~/lyubertssyi',
    'bank_ids': 'all',
    'bank_types': 'all',
    'page': 1,
}

In [185]:
url_current = url_format.format(**url_params)
url_current

'http://www.banki.ru/banks/map/moskva_i_oblast~/lyubertssyi/#/!b1:all!s3:all!s4:list!p1:1'

In [186]:
r = requests.get(url_current).content

In [187]:
# find '<script id="list-template">'

soup = BeautifulSoup(r, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="env-no-js env-desktop" prefix="ya: http://webmaster.yandex.ru/vocabularies/
og: http://ogp.me/ns#" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <title>
   Банкоматы Люберец, список отделений банков в г. Люберцы на карте | Банки.ру
  </title>
  <meta content="банк, Люберцы, банкоматы, филиалы, карта, телефон, режим работы" name="keywords"/>
  <meta content="Актуальный список банкоматов, отделений, офисов и филиалов банков Люберец: адреса, телефоны, режим работы и расположение на карте" name="description"/>
  <meta content="index, follow" name="robots"/>
  <meta content="243196695801552" property="fb:app_id"/>
  <meta content="Банки.ру" property="og:site_name"/>
  <meta content="Банки.ру — сайт о вкладах и банках. Уникальная база данных по вкладам с поисковой системой. Всегда свежие новости по банковской и экономической теме." property="og:description"/>
  <meta co

In [13]:
from selenium import webdriver

import html
from time import sleep

In [14]:
with webdriver.Firefox() as driver:
    driver.get("https://mail.ru/")
    sleep(3)

In [200]:
def get_banks_info(driver, url, t_sleep=1):
    driver.get(url)
    sleep(t_sleep)
    
    result = []
    items = driver.find_elements_by_class_name("list__item")
    
    for item in items:
        classes = item.get_attribute('class').split()
        if 'list__item--header' in classes:
            continue
            
        bank_info = {
            'name': item.get_attribute("data-name"),
            'type': item.get_attribute("data-type"),
            'address': item.get_attribute("data-address"),
            'lat': item.get_attribute("data-latitude"),
            'lon': item.get_attribute("data-longitude"),
            'name_bank': item.find_element_by_class_name("item__name__bank").text,
            'access24h': 'круглосуточно' in
                item.find_element_by_class_name("item__schedule").text.lower(),
        }

        if bank_info['lat'] is not None:
            bank_info['lat'] = float(bank_info['lat'])

        if bank_info['lon'] is not None:
            bank_info['lon'] = float(bank_info['lon'])
            
        for f in ['address', 'name', 'name_bank']:
            if bank_info[f] is None:
                continue
            bank_info[f] = html.unescape(bank_info[f])

        result.append(bank_info)
        
    return result


with webdriver.Firefox() as driver:
    result = get_banks_info(driver, url_current)

In [201]:
result = pd.DataFrame(result)
result.head()

,access24h,address,lat,lon,name,name_bank,type
0,False,"Московская обл., г. Люберцы, ул. Красная, д. 4",55.681925,37.891755,Отделение,Абсолют Банк,office
1,False,"г. Люберцы, ул. Красная, д. 4",55.681923,37.891758,Банкомат,Абсолют Банк,atm
2,False,"Московская обл., г. Люберцы, ул. 3-е Почтовое ...",55.682166,37.868119,Офис «Авангард-Экспресс» № 8835,Авангард,office
3,False,"Московская обл., г. Люберцы, ш. Новорязанское,...",55.669862,37.871511,Офис «Авангард-Экспресс» № 8939,Авангард,office
4,False,"Московская обл., г. Люберцы, ул. Побратимов, д. 7",55.692576,37.896934,Офис «Авангард-Экспресс» № 8944,Авангард,office


In [19]:
import random

def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)

with webdriver.Firefox() as driver:
    driver.get("https://music.yandex.ru/artist/184100")
    random_sleep()
    
    # можно исполнять свой javascript код
    driver.execute_script("window.scrollTo(0, 250);")
    random_sleep(0)
    
    # можно использовать CSS-селекторы или XPath
    # лучше использовать Firefox, он лучше подсказывает пути
    # стоит использовать, если путь выглядит лаконичным
    # пример плохого CSS-селектора
    elem = driver.find_element_by_css_selector(
        'div.d-tabs__tab:nth-child(3) > a:nth-child(1)')
    
    # можно кликать по объектам
    # [ переходим на страничку с альбомами группы ]
    elem.click()
    random_sleep()
    
    # можно искать по классам
    albums = driver.find_element_by_class_name('page-artist__albums').\
        find_elements_by_class_name('album_selectable')
    album = albums[4]
    # [ прокручиваем до нужного альбома ]
    driver.execute_script("return arguments[0].scrollIntoView();", album)
    random_sleep()
    # [ переходим на страничку с выбранным альбомом ]
    album.find_element_by_class_name('d-link').click()
    random_sleep()
    
    driver.execute_script("window.scrollTo(0, 340);")
    random_sleep(0)
    
    # [ кликаем на play ]
    play = driver.find_element_by_class_name('button-play__type_album')
    play.click()
    
    # [ прокликиваем на 5 трек ]
    # пример хорошего CSS-селектора
    next_ = driver.find_element_by_css_selector('.d-icon_track-next')
    for i in range(4):
        next_.click()
        random_sleep(1, 1)
        
    # [ выводим информацию о треке ]
    # пример хорошего CSS-селектора
    elem = driver.find_element_by_css_selector('.track__name')
    artist = elem.find_element_by_class_name('track__artists').text
    title = elem.find_element_by_class_name('track__title').text
    
    tracks = driver.find_element_by_css_selector('.page-album__tracks').\
        find_elements_by_class_name('d-track')
    
    for track_i, track in enumerate(tracks, 1):
        if 'd-track_playing' in track.get_attribute("class").split(' '):
            break
    
    print("Now playing: {} - {}".format(artist, title))
    print("{}th track of {} tracks".format(track_i, len(tracks)))
    
    sleep(15)

Now playing:  - 
4th track of 13 tracks


## Selenium — инструмент тестирования

[Katalon Recorder](https://www.katalon.com/) (Selenium IDE): [Chrome Extension](https://chrome.google.com/webstore/detail/katalon-recorder-selenium/ljdobmomdgdljniojadhoplhkpialdid), [Firefox Extension](https://addons.mozilla.org/en-US/firefox/addon/katalon-automation-record/)

chrome://extensions/

https://mail.ru/ --> "кто такой филантроп?"